In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random

In [ ]:
#Fig 7(a)

def random_immune_nodes(Nodes, J):
    if Nodes == 0:
        selected_elements = []
    else:
        selected_elements = random.sample(list(J.nodes), Nodes)
    return selected_elements


def highest_degree(Nodes, K):
    dictionary = {node: K.degree(node) for node in K.nodes}
    highest_m = heapq.nlargest(Nodes, dictionary.values())
    highest_values_dict = {key: value for key, value in dictionary.items() if value in highest_m}
    highest_list = [node for node in highest_values_dict]
    if len(highest_list) > Nodes:
        selected_elements = random.sample(highest_list, Nodes)
    else:
        selected_elements = highest_list
    
    return selected_elements

def lowest_degree(Nodes, L):
    dictionary = {node: L.degree(node) for node in L.nodes}
    lowest_m = heapq.nsmallest(Nodes, dictionary.values())
    lowest_values_dict = {key: value for key, value in dictionary.items() if value in lowest_m}
    lowest_list = [node for node in lowest_values_dict]
    if len(lowest_list) > Nodes:
        selected_elements = random.sample(lowest_list, Nodes)
    else:
        selected_elements = lowest_list
    return selected_elements   
    

def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict


def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha):
################################ Cascading failure ######################################

    H.remove_node(random.choice([i for i in list(H.nodes)])) #randomly removing a node, avoiding the immume_nodes, to initiate the cascade

    not_failed_nodes =[]
    failed_nodes =[]
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        for each in current_failed_nodes:
            if each in immune_nodes:
                pass
            else :
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
                    
        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}

        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                count +=1
        if count == len(not_failed_nodes):
            break
         
    return failed_nodes


N_nodes = 30000
# p = 0.008
m =120000
realizations = 150
average_degree_of_original_graph = (2*m)/N_nodes 
Phi = np.arange(0.85,0.90,0.005)


suriviving_prob_array_proposed = []
suriviving_prob_array_random = []
suriviving_prob_array_highest_degree = []
suriviving_prob_array_lowest_degree = []

immune_node_size_proposed =[] 
immune_node_size_random =[]
immune_node_size_highest_degree =[] 
immune_node_size_lowest_degree =[] 


for thetha in Phi:
    intermediate_suriviving_prob_array_proposed = []
    intermediate_suriviving_prob_array_random = []
    intermediate_suriviving_prob_array_highest_degree = []
    intermediate_suriviving_prob_array_lowest_degree = []
    
    intermediate_immune_node_size_proposed =[]
    intermediate_immune_node_size_random = []
    intermediate_immune_node_size_highest_degree =[] 
    intermediate_immune_node_size_lowest_degree =[] 
    for i in range(realizations):
        G = nx.gnm_random_graph(N_nodes,m)
        degree_sequence_erdos = list(G.degree())
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])


        remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
        G.remove_nodes_from(remove)

        N_nodes_new = list(G.nodes)    #list of nodes with k>0


        H = G.copy()   # graph contains nodes that have k > 0
        J = G.copy()
        I = G.copy()
        K = G.copy()
        L = G.copy()
        degree_sequence_erdos = list(H.degree())

        selected_nodes = []
        for node in H.nodes:
            neighbor_list = [n for n in H.neighbors(node)]
            selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#         print(selected_nodes)
#         print(H.degree())

        color_dict, avg_color_degrees_dict = avg_color_degree(H)
#         print(avg_color_degrees_dict)

        immune_nodes_proposed = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
        immune_nodes_random = random_immune_nodes(len(immune_nodes_proposed),J)
        immune_nodes_highest_degree = highest_degree(len(immune_nodes_proposed),K)
        immune_nodes_lowest_degree = lowest_degree(len(immune_nodes_proposed),L)

        failed_nodes_at_the_end_cascade_proposed = cascading_failure(G,H,immune_nodes_proposed,thetha)
        failed_nodes_at_the_end_cascade_random = cascading_failure(G,J,immune_nodes_random,thetha)
        failed_nodes_at_the_end_cascade_highest_degree = cascading_failure(G,K,immune_nodes_highest_degree,thetha)
        failed_nodes_at_the_end_cascade_lowest_degree = cascading_failure(G,L,immune_nodes_lowest_degree,thetha)

        suriviving_prob_proposed = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_proposed))/ len(I.nodes)
        suriviving_prob_random = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_random))/ len(I.nodes)
        suriviving_prob_highest_degree = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_highest_degree))/ len(I.nodes)
        suriviving_prob_lowest_degree = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_lowest_degree))/ len(I.nodes)
        
        
        intermediate_suriviving_prob_array_proposed.append(suriviving_prob_proposed)
        intermediate_suriviving_prob_array_random.append(suriviving_prob_random)
        intermediate_suriviving_prob_array_highest_degree.append(suriviving_prob_highest_degree)
        intermediate_suriviving_prob_array_lowest_degree.append(suriviving_prob_lowest_degree)
        
        intermediate_immune_node_size_proposed.append(len(immune_nodes_proposed)/len(N_nodes_new))
        intermediate_immune_node_size_random.append(len(immune_nodes_random)/len(N_nodes_new))
        intermediate_immune_node_size_highest_degree.append(len(immune_nodes_highest_degree)/len(N_nodes_new))
        intermediate_immune_node_size_lowest_degree.append(len(immune_nodes_lowest_degree)/len(N_nodes_new))

        
    suriviving_prob_array_proposed.append(round(np.mean(intermediate_suriviving_prob_array_proposed),2))
    suriviving_prob_array_random.append(round(np.mean(intermediate_suriviving_prob_array_random),2))
    suriviving_prob_array_highest_degree.append(round(np.mean(intermediate_suriviving_prob_array_highest_degree),2))
    suriviving_prob_array_lowest_degree.append(round(np.mean(intermediate_suriviving_prob_array_lowest_degree),2))

    immune_node_size_proposed.append(np.mean(intermediate_immune_node_size_proposed))
    immune_node_size_random.append(np.mean(intermediate_immune_node_size_random))
    immune_node_size_highest_degree.append(np.mean(intermediate_immune_node_size_highest_degree))
    immune_node_size_lowest_degree.append(np.mean(intermediate_immune_node_size_lowest_degree))

print("##### Survival Probabilities ######")    
print("Proposed =",suriviving_prob_array_proposed)
print("Random = ",suriviving_prob_array_random)
print("Highest = ",suriviving_prob_array_highest_degree)
print("Lowest = ",suriviving_prob_array_lowest_degree)

print("###### Immune Node size #######")
print("Proposed =",immune_node_size_proposed)
print("Random = ",immune_node_size_random)
print("Highest = ",immune_node_size_highest_degree)
print("Lowest = ",immune_node_size_lowest_degree)

plt.plot(Phi, suriviving_prob_array_proposed, label = "Proposed Protection")
plt.plot(Phi, suriviving_prob_array_random, label = "Random Nodes")
plt.plot(Phi, suriviving_prob_array_highest_degree, label = "Highest Degree")
plt.plot(Phi, suriviving_prob_array_lowest_degree, label = "Lowest Degree")
# plt.plot(Phi,immune_node_size,'--', label = "immune" )
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.title("ER Network  "+"N = "+str(N_nodes)+" , " "<k> = "+str(average_degree_of_original_graph))
plt.show()


In [ ]:
#Fig 7(b)

def closeness_centrality(Nodes, J):
    closeness_centralities = nx.closeness_centrality(J)
    top_m_nodes = sorted(closeness_centralities, key=closeness_centralities.get, reverse=True)[:Nodes]
    return top_m_nodes

def katz_centrality(Nodes, K):
    katz_centralities = nx.katz_centrality_numpy(K)
    top_m_nodes = sorted(katz_centralities, key=katz_centralities.get, reverse=True)[:Nodes]
    return top_m_nodes

def betweenness_centrality(Nodes, L):
    betweenness_centralities = nx.betweenness_centrality(L)
    top_m_nodes = sorted(betweenness_centralities, key=betweenness_centralities.get, reverse=True)[:Nodes]    
    return top_m_nodes  
   

def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
   
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict


def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha):
################################ Cascading failure ######################################

    H.remove_node(random.choice([i for i in list(H.nodes)])) #randomly removing a node, avoiding the immume_nodes, to initiate the cascade

    not_failed_nodes =[]
    failed_nodes =[]
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        for each in current_failed_nodes:
            if each in immune_nodes:
                pass
            else :
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
                   
        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}

        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                count +=1
        if count == len(not_failed_nodes):
            break
         
    return failed_nodes


N_nodes = 30000
# p = 0.008
m =120000
realizations = 150
average_degree_of_original_graph = (2*m)/N_nodes
Phi = np.arange(0.85,0.90,0.005)


suriviving_prob_array_proposed = []
suriviving_prob_array_closeness_centrality = []
suriviving_prob_array_katz_centrality = []
suriviving_prob_array_betweenness_centrality = []

immune_node_size_proposed =[]
immune_node_size_closeness_centrality =[]
immune_node_size_katz_centrality =[]
immune_node_size_betweenness_centrality =[]


for thetha in Phi:
    intermediate_suriviving_prob_array_proposed = []
    intermediate_suriviving_prob_array_closeness_centrality = []
    intermediate_suriviving_prob_array_katz_centrality = []
    intermediate_suriviving_prob_array_betweenness_centrality = []
   
    intermediate_immune_node_size_proposed =[]
    intermediate_immune_node_size_closeness_centrality = []
    intermediate_immune_node_size_katz_centrality =[]
    intermediate_immune_node_size_betweenness_centrality =[]
    for i in range(realizations):
        G = nx.gnm_random_graph(N_nodes,m)
        degree_sequence_erdos = list(G.degree())
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])


        remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
        G.remove_nodes_from(remove)

        N_nodes_new = list(G.nodes)    #list of nodes with k>0


        H = G.copy()   # graph contains nodes that have k > 0
        J = G.copy()
        I = G.copy()
        K = G.copy()
        L = G.copy()
        degree_sequence_erdos = list(H.degree())

        selected_nodes = []
        for node in H.nodes:
            neighbor_list = [n for n in H.neighbors(node)]
            selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#         print(selected_nodes)
#         print(H.degree())

        color_dict, avg_color_degrees_dict = avg_color_degree(H)
#         print(avg_color_degrees_dict)

        immune_nodes_proposed = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
        immune_nodes_closeness_centrality = closeness_centrality(len(immune_nodes_proposed),J)
        immune_nodes_katz_centrality = katz_centrality(len(immune_nodes_proposed),K)
        immune_nodes_betweenness_centrality = betweenness_centrality(len(immune_nodes_proposed),L)

        failed_nodes_at_the_end_cascade_proposed = cascading_failure(G,H,immune_nodes_proposed,thetha)
        failed_nodes_at_the_end_cascade_closeness_centrality = cascading_failure(G,J,immune_nodes_closeness_centrality,thetha)
        failed_nodes_at_the_end_cascade_katz_centrality = cascading_failure(G,K,immune_nodes_katz_centrality,thetha)
        failed_nodes_at_the_end_cascade_betweenness_centrality = cascading_failure(G,L,immune_nodes_betweenness_centrality,thetha)

        suriviving_prob_proposed = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_proposed))/ len(I.nodes)
        suriviving_prob_closeness_centrality = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_closeness_centrality))/ len(I.nodes)
        suriviving_prob_katz_centrality = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_katz_centrality))/ len(I.nodes)
        suriviving_prob_betweenness_centrality = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_betweenness_centrality))/ len(I.nodes)
       
       
        intermediate_suriviving_prob_array_proposed.append(suriviving_prob_proposed)
        intermediate_suriviving_prob_array_closeness_centrality.append(suriviving_prob_closeness_centrality)
        intermediate_suriviving_prob_array_katz_centrality.append(suriviving_prob_katz_centrality)
        intermediate_suriviving_prob_array_betweenness_centrality.append(suriviving_prob_betweenness_centrality)
       
        intermediate_immune_node_size_proposed.append(len(immune_nodes_proposed)/len(N_nodes_new))
        intermediate_immune_node_size_closeness_centrality.append(len(immune_nodes_closeness_centrality)/len(N_nodes_new))
        intermediate_immune_node_size_katz_centrality.append(len(immune_nodes_katz_centrality)/len(N_nodes_new))
        intermediate_immune_node_size_betweenness_centrality.append(len(immune_nodes_betweenness_centrality)/len(N_nodes_new))

       
    suriviving_prob_array_proposed.append(round(np.mean(intermediate_suriviving_prob_array_proposed),2))
    suriviving_prob_array_closeness_centrality.append(round(np.mean(intermediate_suriviving_prob_array_closeness_centrality),2))
    suriviving_prob_array_katz_centrality.append(round(np.mean(intermediate_suriviving_prob_array_katz_centrality),2))
    suriviving_prob_array_betweenness_centrality.append(round(np.mean(intermediate_suriviving_prob_array_betweenness_centrality),2))

    immune_node_size_proposed.append(np.mean(intermediate_immune_node_size_proposed))
    immune_node_size_closeness_centrality.append(np.mean(intermediate_immune_node_size_closeness_centrality))
    immune_node_size_katz_centrality.append(np.mean(intermediate_immune_node_size_katz_centrality))
    immune_node_size_betweenness_centrality.append(np.mean(intermediate_immune_node_size_betweenness_centrality))

print("##### Survival Probabilities ######")    
print("Proposed =",suriviving_prob_array_proposed)
print("Random = ",suriviving_prob_array_closeness_centrality)
print("Highest = ",suriviving_prob_array_katz_centrality)
print("Lowest = ",suriviving_prob_array_betweenness_centrality)

print("###### Immune Node size #######")
print("Proposed =",immune_node_size_proposed)
print("Random = ",immune_node_size_closeness_centrality)
print("Highest = ",immune_node_size_katz_centrality)
print("Lowest = ",immune_node_size_betweenness_centrality)

plt.plot(Phi, suriviving_prob_array_proposed, label = "Proposed Protection")
plt.plot(Phi, suriviving_prob_array_closeness_centrality, label = "Closeness Centrality")
plt.plot(Phi, suriviving_prob_array_katz_centrality, label = "Katz Centrality")
plt.plot(Phi, suriviving_prob_array_betweenness_centrality, label = "Betweenness Centrality")
# plt.plot(Phi,immune_node_size,'--', label = "immune" )
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.title("ER Network  "+"N = "+str(N_nodes)+" , " "<k> = "+str(average_degree_of_original_graph))
plt.show()



In [ ]:
#Fig 7(c)

def random_immune_nodes(Nodes, J):
    if Nodes == 0:
        selected_elements = []
    else:
        selected_elements = random.sample(list(J.nodes), Nodes)
    return selected_elements


def highest_degree(Nodes, K):
    dictionary = {node: K.degree(node) for node in K.nodes}
    highest_m = heapq.nlargest(Nodes, dictionary.values())
    highest_values_dict = {key: value for key, value in dictionary.items() if value in highest_m}
    highest_list = [node for node in highest_values_dict]
    if len(highest_list) > Nodes:
        selected_elements = random.sample(highest_list, Nodes)
    else:
        selected_elements = highest_list
    
    return selected_elements

def lowest_degree(Nodes, L):
    dictionary = {node: L.degree(node) for node in L.nodes}
    lowest_m = heapq.nsmallest(Nodes, dictionary.values())
    lowest_values_dict = {key: value for key, value in dictionary.items() if value in lowest_m}
    lowest_list = [node for node in lowest_values_dict]
    if len(lowest_list) > Nodes:
        selected_elements = random.sample(lowest_list, Nodes)
    else:
        selected_elements = lowest_list
    return selected_elements   
    

def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict


def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha):
################################ Cascading failure ######################################

    H.remove_node(random.choice([i for i in list(H.nodes)])) #randomly removing a node, avoiding the immume_nodes, to initiate the cascade

    not_failed_nodes =[]
    failed_nodes =[]
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        for each in current_failed_nodes:
            if each in immune_nodes:
                pass
            else :
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
                    
        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}

        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                count +=1
        if count == len(not_failed_nodes):
            break
         
    return failed_nodes


N_nodes = 30000
realizations = 150
average_degree_of_original_graph = (2*m)/N_nodes 
Phi = np.arange(0.85,0.90,0.005)


suriviving_prob_array_proposed = []
suriviving_prob_array_random = []
suriviving_prob_array_highest_degree = []
suriviving_prob_array_lowest_degree = []

immune_node_size_proposed =[] 
immune_node_size_random =[]
immune_node_size_highest_degree =[] 
immune_node_size_lowest_degree =[] 


for thetha in Phi:
    intermediate_suriviving_prob_array_proposed = []
    intermediate_suriviving_prob_array_random = []
    intermediate_suriviving_prob_array_highest_degree = []
    intermediate_suriviving_prob_array_lowest_degree = []
    
    intermediate_immune_node_size_proposed =[]
    intermediate_immune_node_size_random = []
    intermediate_immune_node_size_highest_degree =[] 
    intermediate_immune_node_size_lowest_degree =[] 
    for i in range(realizations):
        G = nx.barabasi_albert_graph(N_nodes,3)
        degree_sequence_erdos = list(G.degree())
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])


        remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
        G.remove_nodes_from(remove)

        N_nodes_new = list(G.nodes)    #list of nodes with k>0


        H = G.copy()   # graph contains nodes that have k > 0
        J = G.copy()
        I = G.copy()
        K = G.copy()
        L = G.copy()
        degree_sequence_erdos = list(H.degree())

        selected_nodes = []
        for node in H.nodes:
            neighbor_list = [n for n in H.neighbors(node)]
            selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#         print(selected_nodes)
#         print(H.degree())

        color_dict, avg_color_degrees_dict = avg_color_degree(H)
#         print(avg_color_degrees_dict)

        immune_nodes_proposed = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
        immune_nodes_random = random_immune_nodes(len(immune_nodes_proposed),J)
        immune_nodes_highest_degree = highest_degree(len(immune_nodes_proposed),K)
        immune_nodes_lowest_degree = lowest_degree(len(immune_nodes_proposed),L)

        failed_nodes_at_the_end_cascade_proposed = cascading_failure(G,H,immune_nodes_proposed,thetha)
        failed_nodes_at_the_end_cascade_random = cascading_failure(G,J,immune_nodes_random,thetha)
        failed_nodes_at_the_end_cascade_highest_degree = cascading_failure(G,K,immune_nodes_highest_degree,thetha)
        failed_nodes_at_the_end_cascade_lowest_degree = cascading_failure(G,L,immune_nodes_lowest_degree,thetha)

        suriviving_prob_proposed = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_proposed))/ len(I.nodes)
        suriviving_prob_random = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_random))/ len(I.nodes)
        suriviving_prob_highest_degree = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_highest_degree))/ len(I.nodes)
        suriviving_prob_lowest_degree = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_lowest_degree))/ len(I.nodes)
        
        
        intermediate_suriviving_prob_array_proposed.append(suriviving_prob_proposed)
        intermediate_suriviving_prob_array_random.append(suriviving_prob_random)
        intermediate_suriviving_prob_array_highest_degree.append(suriviving_prob_highest_degree)
        intermediate_suriviving_prob_array_lowest_degree.append(suriviving_prob_lowest_degree)
        
        intermediate_immune_node_size_proposed.append(len(immune_nodes_proposed)/len(N_nodes_new))
        intermediate_immune_node_size_random.append(len(immune_nodes_random)/len(N_nodes_new))
        intermediate_immune_node_size_highest_degree.append(len(immune_nodes_highest_degree)/len(N_nodes_new))
        intermediate_immune_node_size_lowest_degree.append(len(immune_nodes_lowest_degree)/len(N_nodes_new))

        
    suriviving_prob_array_proposed.append(round(np.mean(intermediate_suriviving_prob_array_proposed),2))
    suriviving_prob_array_random.append(round(np.mean(intermediate_suriviving_prob_array_random),2))
    suriviving_prob_array_highest_degree.append(round(np.mean(intermediate_suriviving_prob_array_highest_degree),2))
    suriviving_prob_array_lowest_degree.append(round(np.mean(intermediate_suriviving_prob_array_lowest_degree),2))

    immune_node_size_proposed.append(np.mean(intermediate_immune_node_size_proposed))
    immune_node_size_random.append(np.mean(intermediate_immune_node_size_random))
    immune_node_size_highest_degree.append(np.mean(intermediate_immune_node_size_highest_degree))
    immune_node_size_lowest_degree.append(np.mean(intermediate_immune_node_size_lowest_degree))

print("##### Survival Probabilities ######")    
print("Proposed =",suriviving_prob_array_proposed)
print("Random = ",suriviving_prob_array_random)
print("Highest = ",suriviving_prob_array_highest_degree)
print("Lowest = ",suriviving_prob_array_lowest_degree)

print("###### Immune Node size #######")
print("Proposed =",immune_node_size_proposed)
print("Random = ",immune_node_size_random)
print("Highest = ",immune_node_size_highest_degree)
print("Lowest = ",immune_node_size_lowest_degree)

plt.plot(Phi, suriviving_prob_array_proposed, label = "Proposed Protection")
plt.plot(Phi, suriviving_prob_array_random, label = "Random Nodes")
plt.plot(Phi, suriviving_prob_array_highest_degree, label = "Highest Degree")
plt.plot(Phi, suriviving_prob_array_lowest_degree, label = "Lowest Degree")
# plt.plot(Phi,immune_node_size,'--', label = "immune" )
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.show()



In [ ]:
#Fig 7(d)

def closeness_centrality(Nodes, J):
    closeness_centralities = nx.closeness_centrality(J)
    top_m_nodes = sorted(closeness_centralities, key=closeness_centralities.get, reverse=True)[:Nodes]
    return top_m_nodes

def katz_centrality(Nodes, K):
    katz_centralities = nx.katz_centrality_numpy(K)
    top_m_nodes = sorted(katz_centralities, key=katz_centralities.get, reverse=True)[:Nodes]
    return top_m_nodes

def betweenness_centrality(Nodes, L):
    betweenness_centralities = nx.betweenness_centrality(L)
    top_m_nodes = sorted(betweenness_centralities, key=betweenness_centralities.get, reverse=True)[:Nodes]    
    return top_m_nodes  
   

def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
   
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict


def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha):
################################ Cascading failure ######################################

    H.remove_node(random.choice([i for i in list(H.nodes)])) #randomly removing a node, avoiding the immume_nodes, to initiate the cascade

    not_failed_nodes =[]
    failed_nodes =[]
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        for each in current_failed_nodes:
            if each in immune_nodes:
                pass
            else :
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
                   
        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}

        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                count +=1
        if count == len(not_failed_nodes):
            break
         
    return failed_nodes


N_nodes = 30000
realizations = 150
average_degree_of_original_graph = (2*m)/N_nodes
Phi = np.arange(0.85,0.90,0.005)


suriviving_prob_array_proposed = []
suriviving_prob_array_closeness_centrality = []
suriviving_prob_array_katz_centrality = []
suriviving_prob_array_betweenness_centrality = []

immune_node_size_proposed =[]
immune_node_size_closeness_centrality =[]
immune_node_size_katz_centrality =[]
immune_node_size_betweenness_centrality =[]


for thetha in Phi:
    intermediate_suriviving_prob_array_proposed = []
    intermediate_suriviving_prob_array_closeness_centrality = []
    intermediate_suriviving_prob_array_katz_centrality = []
    intermediate_suriviving_prob_array_betweenness_centrality = []
   
    intermediate_immune_node_size_proposed =[]
    intermediate_immune_node_size_closeness_centrality = []
    intermediate_immune_node_size_katz_centrality =[]
    intermediate_immune_node_size_betweenness_centrality =[]
    for i in range(realizations):
        G = nx.barabasi_albert_graph(N_nodes,3)
        degree_sequence_erdos = list(G.degree())
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])


        remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
        G.remove_nodes_from(remove)

        N_nodes_new = list(G.nodes)    #list of nodes with k>0


        H = G.copy()   # graph contains nodes that have k > 0
        J = G.copy()
        I = G.copy()
        K = G.copy()
        L = G.copy()
        degree_sequence_erdos = list(H.degree())

        selected_nodes = []
        for node in H.nodes:
            neighbor_list = [n for n in H.neighbors(node)]
            selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#         print(selected_nodes)
#         print(H.degree())

        color_dict, avg_color_degrees_dict = avg_color_degree(H)
#         print(avg_color_degrees_dict)

        immune_nodes_proposed = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
        immune_nodes_closeness_centrality = closeness_centrality(len(immune_nodes_proposed),J)
        immune_nodes_katz_centrality = katz_centrality(len(immune_nodes_proposed),K)
        immune_nodes_betweenness_centrality = betweenness_centrality(len(immune_nodes_proposed),L)

        failed_nodes_at_the_end_cascade_proposed = cascading_failure(G,H,immune_nodes_proposed,thetha)
        failed_nodes_at_the_end_cascade_closeness_centrality = cascading_failure(G,J,immune_nodes_closeness_centrality,thetha)
        failed_nodes_at_the_end_cascade_katz_centrality = cascading_failure(G,K,immune_nodes_katz_centrality,thetha)
        failed_nodes_at_the_end_cascade_betweenness_centrality = cascading_failure(G,L,immune_nodes_betweenness_centrality,thetha)

        suriviving_prob_proposed = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_proposed))/ len(I.nodes)
        suriviving_prob_closeness_centrality = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_closeness_centrality))/ len(I.nodes)
        suriviving_prob_katz_centrality = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_katz_centrality))/ len(I.nodes)
        suriviving_prob_betweenness_centrality = (len(I.nodes) - len(failed_nodes_at_the_end_cascade_betweenness_centrality))/ len(I.nodes)
       
       
        intermediate_suriviving_prob_array_proposed.append(suriviving_prob_proposed)
        intermediate_suriviving_prob_array_closeness_centrality.append(suriviving_prob_closeness_centrality)
        intermediate_suriviving_prob_array_katz_centrality.append(suriviving_prob_katz_centrality)
        intermediate_suriviving_prob_array_betweenness_centrality.append(suriviving_prob_betweenness_centrality)
       
        intermediate_immune_node_size_proposed.append(len(immune_nodes_proposed)/len(N_nodes_new))
        intermediate_immune_node_size_closeness_centrality.append(len(immune_nodes_closeness_centrality)/len(N_nodes_new))
        intermediate_immune_node_size_katz_centrality.append(len(immune_nodes_katz_centrality)/len(N_nodes_new))
        intermediate_immune_node_size_betweenness_centrality.append(len(immune_nodes_betweenness_centrality)/len(N_nodes_new))

       
    suriviving_prob_array_proposed.append(round(np.mean(intermediate_suriviving_prob_array_proposed),2))
    suriviving_prob_array_closeness_centrality.append(round(np.mean(intermediate_suriviving_prob_array_closeness_centrality),2))
    suriviving_prob_array_katz_centrality.append(round(np.mean(intermediate_suriviving_prob_array_katz_centrality),2))
    suriviving_prob_array_betweenness_centrality.append(round(np.mean(intermediate_suriviving_prob_array_betweenness_centrality),2))

    immune_node_size_proposed.append(np.mean(intermediate_immune_node_size_proposed))
    immune_node_size_closeness_centrality.append(np.mean(intermediate_immune_node_size_closeness_centrality))
    immune_node_size_katz_centrality.append(np.mean(intermediate_immune_node_size_katz_centrality))
    immune_node_size_betweenness_centrality.append(np.mean(intermediate_immune_node_size_betweenness_centrality))

print("##### Survival Probabilities ######")    
print("Proposed =",suriviving_prob_array_proposed)
print("Random = ",suriviving_prob_array_closeness_centrality)
print("Highest = ",suriviving_prob_array_katz_centrality)
print("Lowest = ",suriviving_prob_array_betweenness_centrality)

print("###### Immune Node size #######")
print("Proposed =",immune_node_size_proposed)
print("Random = ",immune_node_size_closeness_centrality)
print("Highest = ",immune_node_size_katz_centrality)
print("Lowest = ",immune_node_size_betweenness_centrality)

plt.plot(Phi, suriviving_prob_array_proposed, label = "Proposed Protection")
plt.plot(Phi, suriviving_prob_array_closeness_centrality, label = "Closeness Centrality")
plt.plot(Phi, suriviving_prob_array_katz_centrality, label = "Katz Centrality")
plt.plot(Phi, suriviving_prob_array_betweenness_centrality, label = "Betweenness Centrality")
# plt.plot(Phi,immune_node_size,'--', label = "immune" )
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.show()

